In [2]:
#read datafolder from Google Drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
data_folder = '/content/drive/My Drive/reuters21578/'


sgml_number_of_files = 21
sgml_file_name_template = 'reut2-NNN.sgm'

# Category files
category_files = {
    'to_': ('Topics', 'all-topics-strings.lc.txt'),
    'pl_': ('Places', 'all-places-strings.lc.txt'),
    'pe_': ('People', 'all-people-strings.lc.txt'),
    'or_': ('Organizations', 'all-orgs-strings.lc.txt'),
    'ex_': ('Exchanges', 'all-exchanges-strings.lc.txt')
}

Mounted at /content/drive


In [0]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [0]:
def getkey(x):
    if x=='to_':
        return 'Topics'
    elif x=='pl_':
        return 'Places'
    elif x=='pe_':
        return 'People'
    elif x=='or_':
        return 'Organizations'
    elif x=='ex_':
        return 'Exchanges'
    
    
    
# Create category dataframe

# Read all categories
category_data = []
category_dictionary={'Topics':[],'Places':[],'People':[],'Organizations':[],'Exchanges':[]}
for category_prefix in category_files.keys():
    with open(data_folder + category_files[category_prefix][1], 'r') as file:
        for category in file.readlines():
            category_data.append([category_prefix + category.strip().lower(), 
                                  category_files[category_prefix][0]])

# Create category dataframe
for i in category_data:
#     print(i[1])
    category_dictionary[i[1]].append(i[0].split('_')[1])
news_categories = pd.DataFrame(data=category_data)

# print "category_data: ", category_data
#(news_categories.values).tolist()
# print((category_data[0]))

knn_cats={}

for category_prefix in category_files.keys():
    with open(data_folder + category_files[category_prefix][1], 'r') as file:
        k=getkey(category_prefix)
        knn_cats[k]=file.readlines()
        knn_cats[k]=[s.rstrip('\n') for s in knn_cats[k]]
        strcat=" "
        knn_cats[k]=[strcat.join(knn_cats[k])]


In [5]:
import re
import xml.sax.saxutils as saxutils
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import nltk
nltk.download('stopwords')

Using TensorFlow backend.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
lemmatizer = WordNetLemmatizer()

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;\\n]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def cleanUpSentence(r):#, stop_words = None#
    r = r.lower().replace("<br />", " ")
    r = REPLACE_BY_SPACE_RE.sub(' ', r)
    r = BAD_SYMBOLS_RE.sub('', r)

    r = ' '.join(word for word in r.split() if word not in STOPWORDS)

    words = word_tokenize(r)

    for w in words:
        w = lemmatizer.lemmatize(w)

    return r

In [0]:
# Parse SGML files
def strip_tags(text):
    return re.sub('<[^<]+?>', '', text).strip()

def unescape(text):
    return saxutils.unescape(text)
  
def makeDict(filename, document_X):
  with open(filename, 'rb') as file:

    content = BeautifulSoup(file.read().lower(),'html.parser')

    for newsline in content('reuters'):
      document_categories = []

      document_id = newsline['newid']
      document_body = strip_tags(str(newsline('text')[0].body)).replace('reuter\n&#3;', '')
      if document_body == 'None':
        continue

      doc_categories = strip_tags(str(newsline('topics')[0].body))
      doc_categories = unescape(doc_categories)

      document_body = unescape(document_body)

      document_X[document_id] = document_body
      

def readFiles(test_data = False):
  document_X = {}
  
  if test_data == True:
    file_name = sgml_file_name_template.replace('NNN', '021')
    filename = data_folder + file_name
    makeDict(filename, document_X)
  else:
    for i in range(sgml_number_of_files):
      if i < 10:
        seq = '00' + str(i)
      else:
        seq = '0' + str(i)

      file_name = sgml_file_name_template.replace('NNN', seq)
      print('Reading file: %s' % file_name)
      filename = data_folder + file_name
      makeDict(filename, document_X)
  return document_X


In [8]:
document_X = readFiles()

Reading file: reut2-000.sgm
Reading file: reut2-001.sgm
Reading file: reut2-002.sgm
Reading file: reut2-003.sgm
Reading file: reut2-004.sgm
Reading file: reut2-005.sgm
Reading file: reut2-006.sgm
Reading file: reut2-007.sgm
Reading file: reut2-008.sgm
Reading file: reut2-009.sgm
Reading file: reut2-010.sgm
Reading file: reut2-011.sgm
Reading file: reut2-012.sgm
Reading file: reut2-013.sgm
Reading file: reut2-014.sgm
Reading file: reut2-015.sgm
Reading file: reut2-016.sgm
Reading file: reut2-017.sgm
Reading file: reut2-018.sgm
Reading file: reut2-019.sgm
Reading file: reut2-020.sgm


In [9]:
# data preprocessing
import numpy as np

nltk.download('punkt')
nltk.download('wordnet')
def create_x_matrix(document_X):
    totalX = []
    for i, doc in document_X.items():
        totalX.append(cleanUpSentence(doc))
    max_vocab_size = 200
    input_tokenizer = Tokenizer(200)
    input_tokenizer.fit_on_texts(totalX)
    encoded_docs = input_tokenizer.texts_to_matrix(totalX, mode='count')
    return totalX,encoded_docs

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
totalX,encoded_docs=create_x_matrix(document_X)

In [0]:
print(category_dictionary)

{'Topics': ['acq', 'alum', 'austdlr', 'austral', 'barley', 'bfr', 'bop', 'can', 'carcass', 'castor-meal', 'castor-oil', 'castorseed', 'citruspulp', 'cocoa', 'coconut', 'coconut-oil', 'coffee', 'copper', 'copra-cake', 'corn', 'corn-oil', 'cornglutenfeed', 'cotton', 'cotton-meal', 'cotton-oil', 'cottonseed', 'cpi', 'cpu', 'crude', 'cruzado', 'dfl', 'dkr', 'dlr', 'dmk', 'drachma', 'earn', 'escudo', 'f-cattle', 'ffr', 'fishmeal', 'flaxseed', 'fuel', 'gas', 'gnp', 'gold', 'grain', 'groundnut', 'groundnut-meal', 'groundnut-oil', 'heat', 'hk', 'hog', 'housing', 'income', 'instal-debt', 'interest', 'inventories', 'ipi', 'iron-steel', 'jet', 'jobs', 'l-cattle', 'lead', 'lei', 'lin-meal', 'lin-oil', 'linseed', 'lit', 'livestock', 'lumber', 'lupin', 'meal-feed', 'mexpeso', 'money-fx', 'money-supply', 'naphtha', 'nat-gas', 'nickel', 'nkr', 'nzdlr', 'oat', 'oilseed', 'orange', 'palladium', 'palm-meal', 'palm-oil', 'palmkernel', 'peseta', 'pet-chem', 'platinum', 'plywood', 'pork-belly', 'potato', 'p

In [0]:
def makeOneHotEncoding(totalX):
  words_in_body={}

  for i in range(len(totalX)):
      words=totalX[i].split(' ')
      words_in_body[i]=words    

  one_hot_label=[]
  for key,v in words_in_body.items():
      dict_temp={'Topics':0,'Places':0,'People':0,'Exchanges':0,'Organizations':0}
      for i in v:
          if i in category_dictionary['Topics']:
              dict_temp['Topics']+=1
          if i in category_dictionary['Places']:
              dict_temp['Places']+=1
          if i in category_dictionary['People']:
              dict_temp['People']+=1
          if i in category_dictionary['Exchanges']:
              dict_temp['Exchanges']+=1
          if i in category_dictionary['Organizations']:
              dict_temp['Organizations']+=1

      one_hot_label.append(dict_temp)


  one_hot_label_list = []
  for i in one_hot_label:

      one_hot_label_list.append(list(i.values()))
  return one_hot_label_list,one_hot_label


In [0]:
one_hot_label_list,one_hot_label = makeOneHotEncoding(totalX)

In [13]:
import operator
true_ranks=[]
for i in one_hot_label:
    true_ranks.append(sorted(i.items(),key=operator.itemgetter(1),reverse=True))
print(true_ranks[0])

[('Topics', 7), ('Places', 3), ('People', 0), ('Exchanges', 0), ('Organizations', 0)]


In [0]:
def accuracy_knn(predicted,actual):
    tp=0
    length = len(actual)
    fp=0
    tp_list = []
    for i in range(length):
        if predicted[i][0][0]==actual[i][0][0] and predicted[i][1][0]==actual[i][1][0] and\
          predicted[i][2][0]==actual[i][2][0]:
            tp+=1
            x=tp
        else:
            fp=0
            x=fp
        tp_list.append(x)
           
    
    return tp/float(length),tp_list

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

print(len(totalX))

18583


In [0]:
# tfidf_matrix_train = tfidf_vectorizer.fit_transform(train_set) 
vectorizer = TfidfVectorizer(stop_words='english')
resultsknn=[]
n = 10000
for i in totalX[0:n]:
    sim={}
    for k,j in knn_cats.items():
        sim[k]=(cosine_sim(i,j[0]))
    resultsknn.append(sorted(sim.items(), key=operator.itemgetter(1),reverse=True))

In [20]:
accuracy,tpl=accuracy_knn(resultsknn,true_ranks[:n])
print(accuracy)
print(true_ranks[:n][0])

0.6424
[('Topics', 7), ('Places', 3), ('People', 0), ('Exchanges', 0), ('Organizations', 0)]


In [0]:
true_list = []
pred_list = []
for i in range(n):
  temp_t, temp_p = [], []
  for j in range(5):
    temp_t.append(true_ranks[:n][i][j][0])
    temp_p.append(resultsknn[i][j][0])
  true_list.append(temp_t)
  pred_list.append(temp_p)

In [22]:
my_df  = pd.DataFrame(columns = ['body', 'predicted ranking', 'actual ranking', '_'])
my_df['body'] = totalX[:n]
my_df['predicted ranking'] = pred_list
my_df['actual ranking'] = true_list
my_df['_'] = tpl

my_df = my_df.sort_values(by='_', ascending=False).reset_index(drop=True)
my_df[['body', 'predicted ranking', 'actual ranking']]

,body,predicted ranking,actual ranking
0,four international banks filed claims 1312 mln...,"[Places, Topics, People, Organizations, Exchan...","[Places, Topics, People, Exchanges, Organizati..."
1,shr 22 cts vs 16 cts net 2 460 000 vs 1 730 00...,"[Topics, Places, People, Organizations, Exchan...","[Topics, Places, People, Exchanges, Organizati..."
2,gannett cos usa today said carried total 957 p...,"[Places, Topics, People, Organizations, Exchan...","[Places, Topics, People, Exchanges, Organizati..."
3,dow chemical co believes strong margins chemic...,"[Topics, Places, People, Organizations, Exchan...","[Topics, Places, People, Exchanges, Organizati..."
4,shr loss 177 dlrs vs loss 1047 dlrs net loss 2...,"[Topics, Places, People, Organizations, Exchan...","[Topics, Places, People, Exchanges, Organizati..."
5,commodity futures trading commission cftc said...,"[Topics, Places, People, Organizations, Exchan...","[Topics, Places, People, Exchanges, Organizati..."
6,centronics corp said named salomon inc financi...,"[Topics, Places, People, Organizations, Exchan...","[Topics, Places, People, Exchanges, Organizati..."
7,executive telecommunications inc said board ex...,"[Topics, Places, People, Organizations, Exchan...","[Topics, Places, People, Exchanges, Organizati..."
8,martin marietta corp said signed exclusive agr...,"[People, Topics, Places, Organizations, Exchan...","[People, Topics, Places, Exchanges, Organizati..."
9,gac liquidating trust said completed sale rio ...,"[Topics, Places, People, Organizations, Exchan...","[Topics, Places, People, Exchanges, Organizati..."
